In [1]:
import numpy as np
import pandas as pd
import json
import codecs
import datetime
import os
import sys
sys.path.append('../')
import matplotlib as mpl
import matplotlib.pyplot as plt
from vector_creator.raw_to_df.rawdata_to_df import *
from vector_creator.score_vectors.score_vectors_assembly import *
from vector_creator.stats_models.entropy import *
from vector_creator.preprocess.est_by_df_column import *
from vector_creator.stats_models.auto_regression import *
from vector_creator.preprocess.install_apps_features import *
from vector_creator.stats_models.estimators import *
from vector_creator.score_vectors.vector_indexer import vector_desc_call_logs
from vector_creator.score_vectors.vector_descriptor import *
path = '/Users/smuel/AndroidStudioProjects/LuckyMeIntgratedSDK/metadata_init_phase/'

In [2]:
rdata = list_of_json_files(path)
rdata[0][0]

'05fa68e5-0ba3-4783-9c07-42740c4914c6'

In [3]:
raw = group_metadata(path)
uid  = '05fa68e5-0ba3-4783-9c07-42740c4914c6'
file = '05fa68e5-0ba3-4783-9c07-42740c4914c6_11-06-2021_09-53-01.json'
raw_data = json.load(codecs.open(path + file, 'r', 'utf-8-sig'))

In [4]:
raw_data.keys()

dict_keys(['SamplingConfigurations', 'CallLogs', 'ImgMetaData', 'InstallApps', 'LocationInfo', 'BatteryInfo', 'WifiInfo', 'ScreenInfo'])

In [5]:
loc, df_r = create_df_from_init_metadata(uid, raw_data)

In [6]:
df_r.keys()

dict_keys(['05fa68e5-0ba3-4783-9c07-42740c4914c6_SamplingConfigurations', '05fa68e5-0ba3-4783-9c07-42740c4914c6_CallLogs', '05fa68e5-0ba3-4783-9c07-42740c4914c6_ImgMetaData', '05fa68e5-0ba3-4783-9c07-42740c4914c6_InstallApps', '05fa68e5-0ba3-4783-9c07-42740c4914c6_LocationInfo', '05fa68e5-0ba3-4783-9c07-42740c4914c6_BatteryInfo', '05fa68e5-0ba3-4783-9c07-42740c4914c6_WifiInfo', '05fa68e5-0ba3-4783-9c07-42740c4914c6_ScreenInfo'])

In [7]:
df=df_r['05fa68e5-0ba3-4783-9c07-42740c4914c6_ImgMetaData']

In [8]:
df.head(5)

,IMAGE_TYPE,IMAGE_DATE_TIME
0,image/jpeg,2019-10-16 08:48:55
1,image/jpeg,2019-10-16 08:49:06
2,image/jpeg,2019-10-16 08:49:07
3,image/jpeg,2019-10-20 15:42:48
4,image/jpeg,2019-10-22 14:09:27


In [9]:
loc_tuple = (loc[0]['Latitude'], loc[0]['Longitude'])

In [11]:
import multiprocessing as mp

In [13]:
print(mp.current_process())

<_MainProcess name='MainProcess' parent=None started>


In [10]:
x = score_vector_constructor(path, 4)

(21, 52)


In [25]:
x.iloc[0:5, 0:52]

description,median number photos taken in day - location,mad number photos taken in day - scale,mean number photos taken in day,sample non zero days ratio,mean number of photos in burst,mean number of bursts a week,median night time number photos - location,mad night time number photos - scale,mean night time number photos,sampled non zero nights ratio,...,ratio of weekend and full week bursts a week,ratio of weekdays and full week bursts a week,ar of num of photos in day time lag-1,ar of num of photos in day time lag-3,entropy on number of photos taken in day,mean max ratio of category of installed apps,median max ratio of category of installed apps,app category coverage ratio,entropy of app categories,entropy of install app time of day
05fa68e5-0ba3-4783-9c07-42740c4914c6,2.0,1.482602,4.184971,0.297251,1.711584,1.532468,2.0,1.482602,4.549020,0.090265,...,0.381356,0.466102,0.925484,0.924120,6.535033,0.325397,0.166667,0.375000,4.020844,2.526873
12fa7561-2e77-4334-b9e1-6858ed27570f,3.0,2.965204,27.320755,0.072011,5.382900,0.650485,2.0,1.482602,5.225000,0.062112,...,0.746269,0.268657,5.376335,5.391280,1.900352,0.345455,0.200000,0.196429,3.128085,2.121276
1362474c-0fb5-4932-a33d-5a30f23ac7e3,3.0,2.965204,9.442424,0.069153,1.902320,1.794872,3.0,2.965204,8.955357,0.046940,...,0.480000,0.494286,2.521008,2.398544,5.911040,0.500000,0.500000,0.196429,2.866587,2.478349
148b0326-09e2-404f-9de6-aff6d44f2a38,2.0,1.482602,4.666667,0.652174,1.700405,3.600000,2.0,1.482602,4.109091,0.407407,...,0.388889,0.513889,0.681415,1.056694,5.570877,0.279412,0.125000,0.303571,3.461320,2.483226
17ac0e20-b292-492c-9adc-faf60a068926,4.0,4.447807,17.270270,0.055348,5.605263,0.376963,3.0,2.965204,27.076923,0.029170,...,0.500000,0.472222,1.556856,1.508826,3.365487,0.600000,0.500000,0.178571,2.947703,2.418296


In [12]:
#read the tags from json file for user register non register and create df with tag column
def create_tag_df(json_file_path, column_name): # 'entries.json'
    tag_data = json.load(codecs.open(json_file_path, 'r', 'utf-8-sig'))
    df_tag = pd.DataFrame(tag_data)
    df_tag = df_tag.rename(columns={column_name: ''}) # 'app_userid'
    dftag = df_tag.set_index('')
    return dftag

In [13]:
def norm_df(old_df):
    inxs = old_df.index.values
    zs = zscore(old_df.values, axis=0, ddof=1)
    return pd.DataFrame(zs, columns=old_df.columns, index=inxs)
n_df = norm_df(x)

In [14]:
def norm_df_2(df):
    inxs = df.index.values
    std_scale = preprocessing.StandardScaler().fit(df)
    df_std = std_scale.transform(df)
    return pd.DataFrame(df_std, columns=df.columns, index=inxs)
n2_df = norm_df_2(x)

In [15]:
def minmax_norm(df):
    inxs = df.index.values
    minmax_scale = preprocessing.MinMaxScaler().fit(df)
    df_minmax = minmax_scale.transform(df)
    return pd.DataFrame(df_minmax, columns=df.columns, index=inxs)
mm_df = minmax_norm(x)

In [ ]:
mm_df.max(axis=0)

In [ ]:
n_df.values[:,0].mean()

In [ ]:
dftag = create_tag_df('entries.json', 'app_userid')
dftag

In [ ]:
len(dftag)

In [ ]:
dfjoin = n_df.join(dftag)
dfjoin['User_Register'] = dfjoin['User_Register'].fillna('false')

In [ ]:
len(dfjoin[dfjoin.User_Register == 'false'])

In [ ]:
dfjoin.values[:,0].std(), dfjoin.values[:,0].mean()

In [ ]:
x = pd.read_csv('/Users/smuel/Developer/Python/csv_fles/photogallery_with_tag_for_xgboost.csv')

In [ ]:
x = pd.read_csv('/Users/smuel/Developer/Python/csv_fles/photogallery_with_tag_for_xgboost.csv')


In [ ]:
x.rename(columns={'Unnamed: 0': 'description'}, inplace=True)

In [ ]:
x.head(5)

In [ ]:
x = x.set_index('description')

In [ ]:
len(x)

In [ ]:
len(x[x.User_Register == 1])

In [ ]:
def csv_to_df(path):
    x = pd.read_csv(path)
    x.rename(columns={'Unnamed: 0': 'description'}, inplace=True)
    x = x.set_index('description')
    return x
